## 🔗 Open This Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DavidLangworthy/ds4s/blob/master/Day%201_%20Introduction%20–%20Climate%20Change%20%26%20Basic%20Plotting.ipynb)

# 🌎 Day 1 – Visualizing Global Warming  
### Introduction to Jupyter, Python, and Climate Data

Welcome to Day 1! Today is your first step into data storytelling with Python. You'll get familiar with **Google Colab**, learn how to load and explore a real dataset, and build your **first visualization** — a line chart showing how Earth’s temperature has changed over time.

We’ll focus on one of the most important and well-documented trends in sustainability: **global warming**. By the end of this lab, you’ll be able to build a chart that shows the Earth's average temperature has risen **over 1°C since pre-industrial times**, using NASA climate data.

---

## 🧭 What You'll Learn

Today is all about the **crawl** stage — low stakes, high learning. You'll:

- Get comfortable with **Google Colab** and Jupyter notebook cells
- Learn the basics of **Python and pandas** for loading and inspecting data
- Create your **first Matplotlib line plot**
- Practice **labeling, annotating**, and describing your chart
- Understand how a simple plot can tell a big story

---

## 🔧 Tools & Setup

We’re working in:
- **Google Colab** (runs in your browser — no setup required)
- **Python + pandas** for data handling
- **Matplotlib** for plotting

Everything runs with standard libraries that Colab already has, or you can install with `pip` if needed.

---

## 📊 Dataset: NASA GISTEMP

We'll use real climate data from NASA’s [GISS Surface Temperature Analysis](https://climate.nasa.gov/vital-signs/global-temperature/). This dataset tracks **global surface temperature anomalies** each year — that is, how much warmer or cooler each year is compared to the 1951–1980 average.

The data is simple:
- **One row per year**
- A column with the temperature anomaly (in °C)

That simplicity makes it perfect for our first visualization.

---

## 🛠️ Lab: Visualizing Global Warming

You’ll open the notebook called **“Visualizing Global Warming”**, which will guide you step-by-step through:

### 1. Setting Up
You’ll import `pandas` and `matplotlib.pyplot`, and run a few test cells to make sure everything works.

> 🧠 Hint blocks are included to help you if anything doesn’t run.

### 2. Loading the Data
You’ll load the NASA CSV file into a pandas DataFrame using `pd.read_csv(...)`. You’ll use `.head()` and `.columns` to explore the data.

> Example prompt: “What’s the most recent year in the dataset, and what was the temperature anomaly that year?”

### 3. Creating Your First Plot
You’ll create a **line chart** using `plt.plot(...)`, plotting `Year` on the x-axis and `Temperature Anomaly` on the y-axis. You’ll add:
- A title
- Axis labels
- A horizontal line at 0°C for context

> 🎯 This is the core moment: you’ll see the dramatic upward slope that shows global warming over time.

### 4. Improving the Chart (Optional)
You’ll learn how to:
- Highlight the most recent year with an annotation (e.g., “2024: +1.47°C”)
- Use color and formatting to make the story even clearer

> These are optional stretch goals — but give your chart polish and clarity.

### 5. Interpreting the Story
At the end of the notebook, you’ll write a short description in Markdown:
- What does this chart show?
- What does the trend mean?

> For example: “Global temperatures have steadily risen. The last decade includes the warmest years on record, reaching ~1.47°C above the 20th-century average. This is strong visual evidence of climate change.”

---

## 🧪 Starter vs. Solution

- The **starter notebook** gives you structure, comments, and scaffolded code (`# TODO` prompts).
- The **solution notebook** shows the completed version, in case you need a reference or get stuck.

---

## ✅ By the End of Today

You’ll know how to:
- Use Colab notebooks
- Load a CSV with `pandas`
- Inspect and describe a dataset
- Plot and label a basic chart with `matplotlib`
- Tell a story about climate change using real data

That’s a big first step. Let’s get started — open the notebook and press `Shift + Enter` to run the first cell.


In [ ]:
# Notebook: Day 1 – Climate Change & Basic Plotting
# Load libraries
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# --- Starter Notebook elide: list the imports you need for the analysis ---

# Load NASA GISTEMP global temperature anomaly data
data_dir = Path.cwd() / "data"
temperature_path = data_dir / "GLB.Ts+dSST.csv"

# --- Starter Notebook elide: point to the local CSV file that holds the temperature data ---
df = (
    pd.read_csv(
        temperature_path,
        skiprows=1,
        usecols=[0, 13],
        names=["Year", "TempAnomaly"],
        header=0,
    )
)
df["TempAnomaly"] = pd.to_numeric(df["TempAnomaly"], errors="coerce")

# --- Starter Notebook elide: clean up the dataframe columns and numeric types ---

# Basic inspection
print(df.head())

# --- Starter Notebook elide: inspect the first few rows of the dataframe ---

# Simple line plot
plt.figure(figsize=(10, 5))
plt.plot(df["Year"], df["TempAnomaly"], marker="o", color="red")
plt.axhline(0, color="black", linestyle="--")  # baseline

plt.xlabel("Year")
plt.ylabel("Temperature Anomaly (°C)")
plt.title("Global Temperature Anomaly (1880–2024)")

# Annotation example
plt.annotate(
    "2024 anomaly: {:.2f}°C".format(df["TempAnomaly"].iloc[-1]),
    xy=(df["Year"].iloc[-1], df["TempAnomaly"].iloc[-1]),
    xytext=(2000, 1.2),
    arrowprops=dict(arrowstyle="->", color="black"),
)

plt.show()

# --- Starter Notebook elide: add an annotation that highlights the latest value ---


In [ ]:
# Save the line plot for the course roadmap

from pathlib import Path


def find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "plots").exists() or (candidate / ".git").exists() or (candidate / "README.md").exists():
            return candidate
    return start


repo_root = find_repo_root(Path.cwd())
plots_dir = repo_root / "plots"
plots_dir.mkdir(parents=True, exist_ok=True)

plot_path = plots_dir / "day01_global_temperature_anomaly.png"
plt.gcf().savefig(plot_path, dpi=300, bbox_inches="tight")
print(f"Saved plot to {plot_path.relative_to(repo_root)}")
